## Tutorial: optimizing memory estimation

C-PAC has some built-in heuristics for estimating the memory required for each node, but these estimates can be greatly improved by providing observed memory usage for a given configuration and comparable data. This tutorial will step through the process of an observation run and an optimized run.

Note: The code cells in this notebook are using a [BASH kernel](https://pypi.org/project/bash_kernel) and [cpac (Python package)](https://fcp-indi.github.io/docs/nightly/user/cpac) v0.5.0

### 1. Observation run

First, we need to run a single exemplar through our pipeline configuration to observe how much memory is used at each node. How many cores we provide to this run should be the same number we intend to use in our optimized runs, but we'll want to give this run lots of memory to allow for the built-in conservative memory estimates.

For this tutorial, we'll use a subject from [ADHD-200](http://fcon_1000.projects.nitrc.org/indi/adhd200), from the FCP-INDI AWS S3 bucket, and we'll use the anatomical only preconfiguration.

In [1]:
cpac run \
  s3://fcp-indi/data/Projects/ADHD200/RawDataBIDS/NYU cpac_runs/observation participant \
  --preconfig anat-only \
  --participant_label 0010042 \
  --n_cpus 4 \
  --mem_gb 10

Loading 🐳 Docker[?2004l
Loading 🐳 fcpindi/c-pac:latest as "jclucas (1001)" with these directory bindings:
  local                                       Docker                                      mode
  ------------------------------------------  ------------------------------------------  ------
  /etc/passwd                                 /etc/passwd                                 ro
  /home/jclucas/.cpac                         /home/jclucas/.cpac                         rw
  /home/jclucas/CNL/GitHub/fcp-indi.github.c  /home/jclucas/CNL/GitHub/fcp-indi.github.c  rw
  om/docs/_sources/user/tutorials             om/docs/_sources/user/tutorials
  /home/jclucas/CNL/GitHub/fcp-indi.github.c  /home/jclucas/CNL/GitHub/fcp-indi.github.c  rw
  om/docs/_sources/user/tutorials/cpac_runs/  om/docs/_sources/user/tutorials/cpac_runs/
  observation                                 observation
  /home/jclucas/CNL/GitHub/fcp-indi.github.c  /home/jclucas/CNL/GitHub/fcp-indi.github.c  rw
  om/docs/_s

: 1

We can see the initial estimate requires at least 14.8 GB for some nodes, so we'll need to give it more memory. We have a couple options here.

1. If we don't have hard memory limits (we expect our system will allow us to use more memory than we allocate), we can tell C-PAC to ignore the insufficient resources by adjusting our pipeline configuration, like

`anat-only-insufficient.yml`
```YAML
FROM: anat-only

pipeline_setup:
  system_config:
    raise_insufficient: Off
```

In [2]:
cpac run \
  s3://fcp-indi/data/Projects/ADHD200/RawDataBIDS/NYU cpac_runs/insufficient participant \
  --pipeline_file $PWD/anat-only-insufficient.yml \
  --participant_label 0010042 \
  --n_cpus 4 \
  --mem_gb 10

Loading 🐳 Docker[?2004l
Loading 🐳 fcpindi/c-pac:latest as "jclucas (1001)" with these directory bindings:
  local                                       Docker                                      mode
  ------------------------------------------  ------------------------------------------  ------
  /etc/passwd                                 /etc/passwd                                 ro
  /home/jclucas/.cpac                         /home/jclucas/.cpac                         rw
  /home/jclucas/CNL/GitHub/fcp-indi.github.c  /home/jclucas/CNL/GitHub/fcp-indi.github.c  ro
  om/docs/_sources/user/tutorials/anat-only-  om/docs/_sources/user/tutorials/anat-only-
  insufficient.yml                            insufficient.yml
  /home/jclucas/CNL/GitHub/fcp-indi.github.c  /home/jclucas/CNL/GitHub/fcp-indi.github.c  rw
  om/docs/_sources/user/tutorials             om/docs/_sources/user/tutorials
  /home/jclucas/CNL/GitHub/fcp-indi.github.c  /home/jclucas/CNL/GitHub/fcp-indi.github.c  rw
  om/do

	 [Job 28] Cached (cpac_sub-0010042_ses-1.gather_desc-preproc_T1w.check_for_s3).
220414-15:18:20,479 nipype.workflow INFO:
	 [Job 29] Cached (cpac_sub-0010042_ses-1.gather_from-T1w_to-template_mode-image_desc-linear_xfm.check_for_s3).
220414-15:18:20,482 nipype.workflow INFO:
	 [Job 30] Cached (cpac_sub-0010042_ses-1.gather_label-WM_desc-preproc_mask.check_for_s3).
220414-15:18:20,486 nipype.workflow INFO:
	 [Job 31] Cached (cpac_sub-0010042_ses-1.gather_from-template_to-T1w_mode-image_desc-linear_xfm.check_for_s3).
220414-15:18:20,489 nipype.workflow INFO:
	 [Job 32] Cached (cpac_sub-0010042_ses-1.id_string_from-template_to-T1w_mode-image_desc-linear_xfm_29).
220414-15:18:20,492 nipype.workflow INFO:
	 [Job 36] Cached (cpac_sub-0010042_ses-1.id_string_label-WM_desc-preproc_mask_28).
220414-15:18:20,495 nipype.workflow INFO:
	 [Job 40] Cached (cpac_sub-0010042_ses-1.id_string_from-T1w_to-template_mode-image_desc-linear_xfm_27).
220414-15:18:20,498 nipype.workflow INFO:
	 [Job 44] Cache

	 [Job 102] Cached (cpac_sub-0010042_ses-1.nii_from-T1w_to-template_mode-image_desc-nonlinear_xfm_12).
220414-15:18:23,51 nipype.workflow INFO:
	 [Job 105] Cached (cpac_sub-0010042_ses-1.json_space-T1w_desc-brain_mask_11).
220414-15:18:23,58 nipype.workflow INFO:
	 [Job 106] Cached (cpac_sub-0010042_ses-1.nii_space-T1w_desc-brain_mask_11).
220414-15:18:23,64 nipype.workflow INFO:
	 [Job 109] Cached (cpac_sub-0010042_ses-1.json_label-CSF_mask_10).
220414-15:18:23,70 nipype.workflow INFO:
	 [Job 110] Cached (cpac_sub-0010042_ses-1.nii_label-CSF_mask_10).
220414-15:18:23,75 nipype.workflow INFO:
	 [Job 113] Cached (cpac_sub-0010042_ses-1.anat_template_edge_67).
220414-15:18:24,321 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 23 jobs ready. Free memory (GB): 9.69/10.00, Free processors: 4/4.
220414-15:18:25,45 nipype.workflow INFO:
	 [Job 4] Cached (cpac_sub-0010042_ses-1.bold_scan_params_sub-0010042_ses-1).
220414-15:18:25,54 nipype.workflow INFO:
	 [Job 10] Cached (cpac_sub-0

220414-15:18:25,650 nipype.workflow INFO:
	 [Node] Running "sinker_from-template_to-T1w_mode-image_xfm_18" ("CPAC.utils.interfaces.datasink.DataSink")
220414-15:18:25,817 nipype.workflow INFO:
	 [Node] Finished "cpac_sub-0010042_ses-1.sinker_label-WM_mask_20".
220414-15:18:25,818 nipype.workflow INFO:
	 [Node] Setting-up "cpac_sub-0010042_ses-1.sinker_from-T1w_to-template_mode-image_xfm_17" in "/home/jclucas/CNL/GitHub/fcp-indi.github.com/docs/_sources/user/tutorials/cpac_runs/insufficient/working/cpac_sub-0010042_ses-1/sinker_from-T1w_to-template_mode-image_xfm_17".
220414-15:18:25,821 nipype.workflow INFO:
	 [Node] Finished "cpac_sub-0010042_ses-1.sinker_label-GM_probseg_21".
220414-15:18:25,822 nipype.workflow INFO:
	 [Node] Setting-up "cpac_sub-0010042_ses-1.sinker_label-CSF_probseg_16" in "/home/jclucas/CNL/GitHub/fcp-indi.github.com/docs/_sources/user/tutorials/cpac_runs/insufficient/working/cpac_sub-0010042_ses-1/sinker_label-CSF_probseg_16".
220414-15:18:25,844 nipype.workflow 

                       * cpac_sub-0010042_ses-1.sinker_space-T1w_desc-brain_mask_11
220414-15:18:29,484 nipype.workflow INFO:
	 [Node] Finished "cpac_sub-0010042_ses-1.sinker_space-T1w_desc-brain_mask_11".
220414-15:18:29,485 nipype.workflow INFO:
	 [Node] Finished "cpac_sub-0010042_ses-1.sinker_label-CSF_mask_10".
220414-15:18:29,525 nipype.workflow INFO:
	 [Job 11] Cached (cpac_sub-0010042_ses-1.qc_skullstrip_63.montage_skull.montage_s).
220414-15:18:29,532 nipype.workflow INFO:
	 [Job 12] Cached (cpac_sub-0010042_ses-1.qc_skullstrip_63.montage_skull.montage_a).
220414-15:18:30,317 nipype.workflow INFO:
	 [Job 107] Completed (cpac_sub-0010042_ses-1.sinker_space-T1w_desc-brain_mask_11).
220414-15:18:30,320 nipype.workflow INFO:
	 [Job 111] Completed (cpac_sub-0010042_ses-1.sinker_label-CSF_mask_10).
220414-15:18:30,325 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 9.69/10.00, Free processors: 4/4.
220414-15:18:30,394 nipype.workflow INFO:
	 [

: 1

Or

2. We can just allocate enough memory to satisfy the estimates

In [3]:
cpac run \
  s3://fcp-indi/data/Projects/ADHD200/RawDataBIDS/NYU cpac_runs/enough participant \
  --preconfig anat-only \
  --participant_label 0010042 \
  --n_cpus 4 \
  --mem_gb 14.8

Loading 🐳 Docker[?2004l
Loading 🐳 fcpindi/c-pac:latest as "jclucas (1001)" with these directory bindings:
  local                                       Docker                                      mode
  ------------------------------------------  ------------------------------------------  ------
  /etc/passwd                                 /etc/passwd                                 ro
  /home/jclucas/.cpac                         /home/jclucas/.cpac                         rw
  /home/jclucas/CNL/GitHub/fcp-indi.github.c  /home/jclucas/CNL/GitHub/fcp-indi.github.c  rw
  om/docs/_sources/user/tutorials             om/docs/_sources/user/tutorials
  /home/jclucas/CNL/GitHub/fcp-indi.github.c  /home/jclucas/CNL/GitHub/fcp-indi.github.c  rw
  om/docs/_sources/user/tutorials/cpac_runs/  om/docs/_sources/user/tutorials/cpac_runs/
  enough                                      enough
  /home/jclucas/CNL/GitHub/fcp-indi.github.c  /home/jclucas/CNL/GitHub/fcp-indi.github.c  rw
  om/docs/_source

220414-15:25:40,477 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 35 jobs ready. Free memory (GB): 14.49/14.80, Free processors: 4/4.
220414-15:25:40,556 nipype.workflow INFO:
	 [Job 14] Cached (cpac_sub-0010042_ses-1.montage_mni_anat_67.resample_u).
220414-15:25:42,479 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 34 jobs ready. Free memory (GB): 14.49/14.80, Free processors: 4/4.
220414-15:25:42,563 nipype.workflow INFO:
	 [Job 19] Cached (cpac_sub-0010042_ses-1.gather_label-GM_mask.check_for_s3).
220414-15:25:42,568 nipype.workflow INFO:
	 [Job 20] Cached (cpac_sub-0010042_ses-1.gather_label-CSF_probseg.check_for_s3).
220414-15:25:42,572 nipype.workflow INFO:
	 [Job 21] Cached (cpac_sub-0010042_ses-1.gather_from-T1w_to-template_mode-image_xfm.check_for_s3).
220414-15:25:42,577 nipype.workflow INFO:
	 [Job 22] Cached (cpac_sub-0010042_ses-1.gather_from-template_to-T1w_mode-image_xfm.check_for_s3).
220414-15:25:44,481 nipype.workflow INFO:
	 [MultiProc] Running 0

220414-15:25:57,699 nipype.workflow INFO:
	 [Job 49] Cached (cpac_sub-0010042_ses-1.json_desc-preproc_T1w_25).
220414-15:25:57,705 nipype.workflow INFO:
	 [Job 50] Cached (cpac_sub-0010042_ses-1.nii_desc-preproc_T1w_25).
220414-15:25:59,555 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 19 jobs ready. Free memory (GB): 14.49/14.80, Free processors: 4/4.
220414-15:25:59,641 nipype.workflow INFO:
	 [Node] Setting-up "cpac_sub-0010042_ses-1.sinker_desc-reorient_T1w_26" in "/home/jclucas/CNL/GitHub/fcp-indi.github.com/docs/_sources/user/tutorials/cpac_runs/enough/working/cpac_sub-0010042_ses-1/sinker_desc-reorient_T1w_26".
220414-15:25:59,647 nipype.workflow INFO:
	 [Node] Setting-up "cpac_sub-0010042_ses-1.sinker_desc-preproc_T1w_25" in "/home/jclucas/CNL/GitHub/fcp-indi.github.com/docs/_sources/user/tutorials/cpac_runs/enough/working/cpac_sub-0010042_ses-1/sinker_desc-preproc_T1w_25".
220414-15:25:59,648 nipype.workflow INFO:
	 [Node] Running "sinker_desc-reorient_T1w_26" ("CPA

220414-15:26:11,651 nipype.workflow INFO:
	 [Node] Running "sinker_label-CSF_desc-preproc_mask_19" ("CPAC.utils.interfaces.datasink.DataSink")
220414-15:26:11,653 nipype.workflow INFO:
	 [Job 78] Cached (cpac_sub-0010042_ses-1.nii_from-template_to-T1w_mode-image_xfm_18).
220414-15:26:11,854 nipype.workflow INFO:
	 [Node] Finished "cpac_sub-0010042_ses-1.sinker_label-WM_mask_20".
220414-15:26:11,860 nipype.workflow INFO:
	 [Node] Finished "cpac_sub-0010042_ses-1.sinker_label-CSF_desc-preproc_mask_19".
220414-15:26:13,564 nipype.workflow INFO:
	 [Job 71] Completed (cpac_sub-0010042_ses-1.sinker_label-WM_mask_20).
220414-15:26:13,567 nipype.workflow INFO:
	 [Job 75] Completed (cpac_sub-0010042_ses-1.sinker_label-CSF_desc-preproc_mask_19).
220414-15:26:13,573 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 10 jobs ready. Free memory (GB): 14.49/14.80, Free processors: 4/4.
220414-15:26:13,663 nipype.workflow INFO:
	 [Job 80] Cached (cpac_sub-0010042_ses-1.id_string_from-T1w_to-tem

220414-15:26:25,682 nipype.workflow INFO:
	 [Job 108] Cached (cpac_sub-0010042_ses-1.id_string_label-CSF_mask_10).
220414-15:26:25,684 nipype.workflow INFO:
	 [Node] Running "sinker_from-T1w_to-template_mode-image_desc-nonlinear_xfm_12" ("CPAC.utils.interfaces.datasink.DataSink")
220414-15:26:25,686 nipype.workflow INFO:
	 [Job 112] Cached (cpac_sub-0010042_ses-1.resampled_T1w-brain-template).
220414-15:26:25,896 nipype.workflow INFO:
	 [Node] Finished "cpac_sub-0010042_ses-1.sinker_from-T1w_to-template_mode-image_desc-nonlinear_xfm_12".
220414-15:26:27,573 nipype.workflow INFO:
	 [Job 103] Completed (cpac_sub-0010042_ses-1.sinker_from-T1w_to-template_mode-image_desc-nonlinear_xfm_12).
220414-15:26:27,578 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 5 jobs ready. Free memory (GB): 14.49/14.80, Free processors: 4/4.
220414-15:26:27,650 nipype.workflow INFO:
	 [Job 105] Cached (cpac_sub-0010042_ses-1.json_space-T1w_desc-brain_mask_11).
220414-15:26:27,655 nipype.workflow INFO

: 1

Now we have a `callback.log` file with observed memory usage. We can peek at the observations with `cpac parse-resources`.

In [4]:
cpac parse-resources --help

usage: cpac parse-resources [-h]
                            [--filter_field {runtime,estimate,efficiency}]
                            [--filter_group {lowest,highest}]
                            [--filter_count FILTER_COUNT]
                            callback

positional arguments:
  callback              callback.log file found in the 'log' directory of the
                        specified derivatives path

optional arguments:
  -h, --help            show this help message and exit
  --filter_field {runtime,estimate,efficiency}, -f {runtime,estimate,efficiency}
  --filter_group {lowest,highest}, -g {lowest,highest}
  --filter_count FILTER_COUNT, -n FILTER_COUNT


: 1

Looking at the 5 most efficient nodes in each successful run, we can see the resource usage varies from run to run, even with the same data, system, and configuration:

In [5]:
cpac parse-resources \
  --filter_field efficiency \
  --filter_group highest \
  --filter_count 5 \
  cpac_runs/insufficient/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                          ┃ Memory    ┃ Memory    ┃ Memory    ┃
┃ Task ID                                  ┃ Used      ┃ Estimated ┃ Efficien… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.8167    │ 1.2235    │ 66.75 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.8133    │ 1.2235    │ 66.48 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.7481    │ 1.8547    │ 40.33 %   │
│ cpac_sub-0010042_ses-1.WM_63.seg_tissue… │ 0.6710    │ 1.7051    │ 39.35 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.7274    │ 1.8547    │ 39.22 %   │
└──────────────────────────────────────────┴───────────┴───────────┴───────────┘


: 1

In [6]:
cpac parse-resources \
  --filter_field efficiency \
  --filter_group highest \
  --filter_count 5 \
  cpac_runs/enough/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                          ┃ Memory    ┃ Memory    ┃ Memory    ┃
┃ Task ID                                  ┃ Used      ┃ Estimated ┃ Efficien… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.7283    │ 1.2235    │ 59.53 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.7850    │ 1.8547    │ 42.32 %   │
│ cpac_sub-0010042_ses-1.CSF_64.seg_tissu… │ 0.7159    │ 1.7051    │ 41.99 %   │
│ cpac_sub-0010042_ses-1.WM_64.seg_tissue… │ 0.6658    │ 1.7051    │ 39.05 %   │
│ cpac_sub-0010042_ses-1.GM_64.seg_tissue… │ 0.6486    │ 1.7051    │ 38.04 %   │
└──────────────────────────────────────────┴───────────┴───────────┴───────────┘


: 1

Let's look at the other bottom five efficiency and the top and bottom five for each of the other fields for one of these `callback.log`s.

In [7]:
cpac parse-resources \
  --filter_field efficiency \
  --filter_group lowest \
  --filter_count 5 \
  cpac_runs/enough/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                          ┃ Memory    ┃ Memory    ┃ Memory    ┃
┃ Task ID                                  ┃ Used      ┃ Estimated ┃ Efficien… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ cpac_sub-0010042_ses-1.montage_mni_anat… │ 0.2135    │ 13.8000   │ 1.55 %    │
│ cpac_sub-0010042_ses-1.anat_reorient_0   │ 0.2233    │ 13.8000   │ 1.62 %    │
│ cpac_sub-0010042_ses-1.qc_skullstrip_63… │ 0.2272    │ 13.8000   │ 1.65 %    │
│ cpac_sub-0010042_ses-1.qc_skullstrip_63… │ 0.2272    │ 13.8000   │ 1.65 %    │
│ cpac_sub-0010042_ses-1.qc_skullstrip_63… │ 0.2272    │ 13.8000   │ 1.65 %    │
└──────────────────────────────────────────┴───────────┴───────────┴───────────┘


: 1

In [8]:
cpac parse-resources \
  --filter_field estimate \
  --filter_group highest \
  --filter_count 5 \
  cpac_runs/enough/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                          ┃ Memory    ┃ Memory    ┃ Memory    ┃
┃ Task ID                                  ┃ Used      ┃ Estimated ┃ Efficien… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ cpac_sub-0010042_ses-1.montage_mni_anat… │ 0.2135    │ 13.8000   │ 1.55 %    │
│ cpac_sub-0010042_ses-1.montage_mni_anat… │ 0.2279    │ 13.8000   │ 1.65 %    │
│ cpac_sub-0010042_ses-1.anat_reorient_0   │ 0.2233    │ 13.8000   │ 1.62 %    │
│ cpac_sub-0010042_ses-1.montage_mni_anat… │ 0.2279    │ 13.8000   │ 1.65 %    │
│ cpac_sub-0010042_ses-1.qc_skullstrip_82… │ 0.2348    │ 13.8000   │ 1.70 %    │
└──────────────────────────────────────────┴───────────┴───────────┴───────────┘


: 1

In [9]:
cpac parse-resources \
  --filter_field runtime \
  --filter_group highest \
  --filter_count 5 \
  cpac_runs/enough/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                          ┃ Memory    ┃ Memory    ┃ Memory    ┃
┃ Task ID                                  ┃ Used      ┃ Estimated ┃ Efficien… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ cpac_sub-0010042_ses-1.segment_64        │ 1.1257    │ 3.4958    │ 32.20 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.9344    │ 3.0513    │ 30.62 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.7850    │ 1.8547    │ 42.32 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.7283    │ 1.2235    │ 59.53 %   │
│ cpac_sub-0010042_ses-1.CSF_64.seg_tissu… │ 0.7159    │ 1.7051    │ 41.99 %   │
└──────────────────────────────────────────┴───────────┴───────────┴───────────┘


: 1

In [10]:
cpac parse-resources \
  --filter_field estimate \
  --filter_group lowest \
  --filter_count 5 \
  cpac_runs/enough/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                          ┃ Memory    ┃ Memory    ┃ Memory    ┃
┃ Task ID                                  ┃ Used      ┃ Estimated ┃ Efficien… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.7283    │ 1.2235    │ 59.53 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.3601    │ 1.2806    │ 28.12 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.2212    │ 1.4200    │ 15.58 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.3732    │ 1.5000    │ 24.88 %   │
│ cpac_sub-0010042_ses-1.CSF_64.seg_tissu… │ 0.7159    │ 1.7051    │ 41.99 %   │
└──────────────────────────────────────────┴───────────┴───────────┴───────────┘


: 1

In [11]:
cpac parse-resources \
  --filter_field runtime \
  --filter_group lowest \
  --filter_count 5 \
  cpac_runs/enough/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                          ┃ Memory    ┃ Memory    ┃ Memory    ┃
┃ Task ID                                  ┃ Used      ┃ Estimated ┃ Efficien… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ cpac_sub-0010042_ses-1.CSF_64.seg_tissu… │ 0.1778    │ 2.5000    │ 7.11 %    │
│ cpac_sub-0010042_ses-1.func_ingress_sub… │ 0.1788    │ 2.0000    │ 8.94 %    │
│ cpac_sub-0010042_ses-1.func_ingress_sub… │ 0.1789    │ 2.0000    │ 8.95 %    │
│ cpac_sub-0010042_ses-1.func_ingress_sub… │ 0.1789    │ 2.0000    │ 8.95 %    │
│ cpac_sub-0010042_ses-1.func_ingress_sub… │ 0.1796    │ 2.0000    │ 8.98 %    │
└──────────────────────────────────────────┴───────────┴───────────┴───────────┘


: 1

### 2. Memory optimized runs

Now that we have observed memory usage specific to our system + configuration + data shape, we can use these observations to inform the estimates of our subesquent runs. Here we'll run another subject and re-run the same intial subject so we can compare the performance. We'll use our original desired memory constraints.

In this example, we'll use a buffer of 25% to demonstrate adjusting the buffer. If you don't set a buffer, the default of 10% will be used. If you don't want a buffer, you can set the buffer to 0.

We can do this with a pipeline configuration like

`anat-only-optimized.yml`
```YAML
FROM: anat-only
pipeline_setup:
  system_config:
    observed_usage:
      callback_log: cpac_runs/enough/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log
      buffer: 25
```

or we can use commandline flags like

In [12]:
cpac run \
  s3://fcp-indi/data/Projects/ADHD200/RawDataBIDS/NYU cpac_runs/optimized participant \
  --preconfig anat-only \
  --participant_label 0010042 5971050 \
  --n_cpus 4 \
  --mem_gb 10 \
  --runtime_usage cpac_runs/enough/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log \
  --runtime_buffer 25

Loading 🐳 Docker[?2004l
Loading 🐳 fcpindi/c-pac:latest as "jclucas (1001)" with these directory bindings:
  local                                       Docker                                      mode
  ------------------------------------------  ------------------------------------------  ------
  /etc/passwd                                 /etc/passwd                                 ro
  /home/jclucas/.cpac                         /home/jclucas/.cpac                         rw
  /home/jclucas/CNL/GitHub/fcp-indi.github.c  /home/jclucas/CNL/GitHub/fcp-indi.github.c  ro
  om/docs/_sources/user/tutorials/cpac_runs/  om/docs/_sources/user/tutorials/cpac_runs/
  enough/log/pipeline_cpac_anat/sub-0010042_  enough/log/pipeline_cpac_anat/sub-0010042_
  ses-1/callback.log                          ses-1/callback.log
  /home/jclucas/CNL/GitHub/fcp-indi.github.c  /home/jclucas/CNL/GitHub/fcp-indi.github.c  rw
  om/docs/_sources/user/tutorials             om/docs/_sources/user/tutorials
  /home/j

220414-15:30:30,194 nipype.workflow INFO:
	 [Job 11] Cached (cpac_sub-0010042_ses-1.qc_skullstrip_63.montage_skull.montage_s).
220414-15:30:30,200 nipype.workflow INFO:
	 [Job 12] Cached (cpac_sub-0010042_ses-1.qc_skullstrip_63.montage_skull.montage_a).
220414-15:30:30,205 nipype.workflow INFO:
	 [Job 18] Cached (cpac_sub-0010042_ses-1.gather_label-GM_desc-preproc_mask.check_for_s3).
220414-15:30:30,210 nipype.workflow INFO:
	 [Job 19] Cached (cpac_sub-0010042_ses-1.gather_label-GM_mask.check_for_s3).
220414-15:30:32,120 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 33 jobs ready. Free memory (GB): 9.69/10.00, Free processors: 4/4.
220414-15:30:32,192 nipype.workflow INFO:
	 [Job 20] Cached (cpac_sub-0010042_ses-1.gather_label-CSF_probseg.check_for_s3).
220414-15:30:32,198 nipype.workflow INFO:
	 [Job 21] Cached (cpac_sub-0010042_ses-1.gather_from-T1w_to-template_mode-image_xfm.check_for_s3).
220414-15:30:32,204 nipype.workflow INFO:
	 [Job 22] Cached (cpac_sub-0010042_ses-1

220414-15:30:46,226 nipype.workflow INFO:
	 [Job 46] Cached (cpac_sub-0010042_ses-1.nii_desc-reorient_T1w_26).
220414-15:30:46,230 nipype.workflow INFO:
	 [Job 48] Cached (cpac_sub-0010042_ses-1.id_string_desc-preproc_T1w_25).
220414-15:30:46,234 nipype.workflow INFO:
	 [Job 52] Cached (cpac_sub-0010042_ses-1.id_string_label-WM_probseg_24).
220414-15:30:46,237 nipype.workflow INFO:
	 [Job 56] Cached (cpac_sub-0010042_ses-1.id_string_space-template_desc-brain_T1w_23).
220414-15:30:48,148 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 21 jobs ready. Free memory (GB): 9.69/10.00, Free processors: 4/4.
220414-15:30:48,222 nipype.workflow INFO:
	 [Job 49] Cached (cpac_sub-0010042_ses-1.json_desc-preproc_T1w_25).
220414-15:30:48,223 nipype.workflow INFO:
	 [Node] Setting-up "cpac_sub-0010042_ses-1.sinker_desc-reorient_T1w_26" in "/home/jclucas/CNL/GitHub/fcp-indi.github.com/docs/_sources/user/tutorials/cpac_runs/optimized/working/cpac_sub-0010042_ses-1/sinker_desc-reorient_T1w_26".

220414-15:31:00,373 nipype.workflow INFO:
	 [Node] Running "sinker_label-WM_mask_20" ("CPAC.utils.interfaces.datasink.DataSink")
220414-15:31:00,374 nipype.workflow INFO:
	 [Job 80] Cached (cpac_sub-0010042_ses-1.id_string_from-T1w_to-template_mode-image_xfm_17).
220414-15:31:00,374 nipype.workflow INFO:
	 [Node] Running "sinker_label-CSF_desc-preproc_mask_19" ("CPAC.utils.interfaces.datasink.DataSink")
220414-15:31:00,587 nipype.workflow INFO:
	 [Node] Finished "cpac_sub-0010042_ses-1.sinker_label-WM_mask_20".
220414-15:31:00,634 nipype.workflow INFO:
	 [Node] Finished "cpac_sub-0010042_ses-1.sinker_label-CSF_desc-preproc_mask_19".
220414-15:31:02,157 nipype.workflow INFO:
	 [Job 71] Completed (cpac_sub-0010042_ses-1.sinker_label-WM_mask_20).
220414-15:31:02,160 nipype.workflow INFO:
	 [Job 75] Completed (cpac_sub-0010042_ses-1.sinker_label-CSF_desc-preproc_mask_19).
220414-15:31:02,165 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 12 jobs ready. Free memory (GB): 9.69/10.0

220414-15:31:14,271 nipype.workflow INFO:
	 [Job 102] Cached (cpac_sub-0010042_ses-1.nii_from-T1w_to-template_mode-image_desc-nonlinear_xfm_12).
220414-15:31:14,277 nipype.workflow INFO:
	 [Job 105] Cached (cpac_sub-0010042_ses-1.json_space-T1w_desc-brain_mask_11).
220414-15:31:14,283 nipype.workflow INFO:
	 [Job 106] Cached (cpac_sub-0010042_ses-1.nii_space-T1w_desc-brain_mask_11).
220414-15:31:16,169 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 4 jobs ready. Free memory (GB): 9.69/10.00, Free processors: 4/4.
220414-15:31:16,274 nipype.workflow INFO:
	 [Node] Setting-up "cpac_sub-0010042_ses-1.sinker_from-T1w_to-template_mode-image_desc-nonlinear_xfm_12" in "/home/jclucas/CNL/GitHub/fcp-indi.github.com/docs/_sources/user/tutorials/cpac_runs/optimized/working/cpac_sub-0010042_ses-1/sinker_from-T1w_to-template_mode-image_desc-nonlinear_xfm_12".
220414-15:31:16,279 nipype.workflow INFO:
	 [Job 108] Cached (cpac_sub-0010042_ses-1.id_string_label-CSF_mask_10).
220414-15:31:16,

...

220414-15:32:12,355 nipype.workflow INFO:
	 [Node] Setting-up "cpac_sub-5971050_ses-1.id_string_desc-reorient_T1w_16" in "/home/jclucas/CNL/GitHub/fcp-indi.github.com/docs/_sources/user/tutorials/cpac_runs/optimized/working/cpac_sub-5971050_ses-1/id_string_desc-reorient_T1w_16".
220414-15:32:12,356 nipype.workflow INFO:
	 [Node] Setting-up "cpac_sub-5971050_ses-1.id_string_label-GM_desc-preproc_mask_15" in "/home/jclucas/CNL/GitHub/fcp-indi.github.com/docs/_sources/user/tutorials/cpac_runs/optimized/working/cpac_sub-5971050_ses-1/id_string_label-GM_desc-preproc_mask_15".
220414-15:32:12,358 nipype.workflow INFO:
	 [Node] Running "id_string_desc-reorient_T1w_16" ("CPAC.utils.interfaces.function.function.Function")
220414-15:32:12,359 nipype.workflow INFO:
	 [Node] Running "id_string_label-GM_desc-preproc_mask_15" ("CPAC.utils.interfaces.function.function.Function")
220414-15:32:12,360 nipype.workflow INFO:
	 [Node] Running "nii_desc-preproc_T1w_17" ("nipype.interfaces.utility.base.Renam

220414-15:32:18,354 nipype.workflow INFO:
	 [Node] Setting-up "cpac_sub-5971050_ses-1.nii_from-T1w_to-template_mode-image_desc-nonlinear_xfm_14" in "/home/jclucas/CNL/GitHub/fcp-indi.github.com/docs/_sources/user/tutorials/cpac_runs/optimized/working/cpac_sub-5971050_ses-1/nii_from-T1w_to-template_mode-image_desc-nonlinear_xfm_14".
220414-15:32:18,359 nipype.workflow INFO:
	 [Node] Running "nii_from-T1w_to-template_mode-image_desc-nonlinear_xfm_14" ("nipype.interfaces.utility.base.Rename")
220414-15:32:18,360 nipype.workflow INFO:
	 [Node] Running "sinker_label-GM_desc-preproc_mask_15" ("CPAC.utils.interfaces.datasink.DataSink")
220414-15:32:18,361 nipype.workflow INFO:
	 [Node] Setting-up "cpac_sub-5971050_ses-1.json_label-GM_probseg_13" in "/home/jclucas/CNL/GitHub/fcp-indi.github.com/docs/_sources/user/tutorials/cpac_runs/optimized/working/cpac_sub-5971050_ses-1/json_label-GM_probseg_13".
220414-15:32:18,370 nipype.workflow INFO:
	 [Node] Running "json_label-GM_probseg_13" ("CPAC.ut

220414-15:32:24,357 nipype.workflow INFO:
	 [Node] Setting-up "cpac_sub-5971050_ses-1.nii_label-GM_mask_11" in "/home/jclucas/CNL/GitHub/fcp-indi.github.com/docs/_sources/user/tutorials/cpac_runs/optimized/working/cpac_sub-5971050_ses-1/nii_label-GM_mask_11".
220414-15:32:24,359 nipype.workflow INFO:
	 [Job 118] Cached (cpac_sub-5971050_ses-1.resampled_T1w-brain-template).
220414-15:32:24,360 nipype.workflow INFO:
	 [Node] Setting-up "cpac_sub-5971050_ses-1.id_string_from-template_to-T1w_mode-image_desc-nonlinear_xfm_10" in "/home/jclucas/CNL/GitHub/fcp-indi.github.com/docs/_sources/user/tutorials/cpac_runs/optimized/working/cpac_sub-5971050_ses-1/id_string_from-template_to-T1w_mode-image_desc-nonlinear_xfm_10".
220414-15:32:24,360 nipype.workflow INFO:
	 [Node] Running "sinker_space-template_desc-brain_T1w_12" ("CPAC.utils.interfaces.datasink.DataSink")
220414-15:32:24,364 nipype.workflow INFO:
	 [Node] Running "nii_label-GM_mask_11" ("nipype.interfaces.utility.base.Rename")
220414-15

220414-15:32:34,265 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 0 jobs ready. Free memory (GB): 9.69/10.00, Free processors: 4/4.
220414-15:32:34,266 nipype.workflow INFO:
	 No tasks are being run, and no jobs can be submitted to the queue. Potential deadlock
220414-15:32:36,837 nipype.workflow WARNING:
	 The following nodes used excessive resources:
---------------------------------------------

cpac_sub-5971050_ses-1
  .sinker_space-T1w_desc-brain_mask_47
      **memory_gb**
        runtime > estimated
        0.239810943359375 > 0.23887157470703124

cpac_sub-5971050_ses-1
  .sinker_desc-brain_T1w_50
      **memory_gb**
        runtime > estimated
        0.23128891015625 > 0.23091316162109377

cpac_sub-5971050_ses-1
  .nii_label-CSF_mask_73
      **memory_gb**
        runtime > estimated
        0.23983383203125 > 0.22700309814453123

cpac_sub-5971050_ses-1
  .sinker_label-GM_probseg_71
      **memory_gb**
        runtime > estimated
        0.23983383203125 > 0.2270030

: 1

Now let's see how our efficiency compares across runs.

In [13]:
cpac parse-resources \
  --filter_field efficiency \
  --filter_group highest \
  --filter_count 5 \
  cpac_runs/optimized/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                          ┃ Memory    ┃ Memory    ┃ Memory    ┃
┃ Task ID                                  ┃ Used      ┃ Estimated ┃ Efficien… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ cpac_sub-0010042_ses-1.sinker_label-GM_… │ 0.2583    │ 0.2270    │ 113.79 %  │
│ cpac_sub-0010042_ses-1.nii_space-templa… │ 0.2554    │ 0.2272    │ 112.40 %  │
│ cpac_sub-0010042_ses-1.sinker_desc-brai… │ 0.2583    │ 0.2309    │ 111.87 %  │
│ cpac_sub-0010042_ses-1.nii_label-WM_des… │ 0.2368    │ 0.2246    │ 105.45 %  │
│ cpac_sub-0010042_ses-1.montage_mni_anat… │ 0.3656    │ 0.3714    │ 98.45 %   │
└──────────────────────────────────────────┴───────────┴───────────┴───────────┘


: 1

In [14]:
cpac parse-resources \
  --filter_field efficiency \
  --filter_group lowest \
  --filter_count 5 \
  cpac_runs/optimized/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                          ┃ Memory    ┃ Memory    ┃ Memory    ┃
┃ Task ID                                  ┃ Used      ┃ Estimated ┃ Efficien… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ cpac_sub-0010042_ses-1.resampled_T1w-br… │ 0.1883    │ 2.0000    │ 9.41 %    │
│ cpac_sub-0010042_ses-1.montage_mni_anat… │ 0.2606    │ 0.5124    │ 50.87 %   │
│ cpac_sub-0010042_ses-1.CSF_64.seg_tissu… │ 0.1830    │ 0.3170    │ 57.74 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.1926    │ 0.3283    │ 58.65 %   │
│ cpac_sub-0010042_ses-1.get_pve_wm_64     │ 0.2566    │ 0.4225    │ 60.73 %   │
└──────────────────────────────────────────┴───────────┴───────────┴───────────┘


: 1

In [15]:
cpac parse-resources \
  --filter_field estimate \
  --filter_group highest \
  --filter_count 5 \
  cpac_runs/optimized/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                          ┃ Memory    ┃ Memory    ┃ Memory    ┃
┃ Task ID                                  ┃ Used      ┃ Estimated ┃ Efficien… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ cpac_sub-0010042_ses-1.resampled_T1w-br… │ 0.1883    │ 2.0000    │ 9.41 %    │
│ cpac_sub-0010042_ses-1.segment_64        │ 1.1376    │ 1.4072    │ 80.84 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.9533    │ 1.1681    │ 81.62 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.7189    │ 0.9812    │ 73.27 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.7630    │ 0.9104    │ 83.81 %   │
└──────────────────────────────────────────┴───────────┴───────────┴───────────┘


: 1

In [16]:
cpac parse-resources \
  --filter_field runtime \
  --filter_group highest \
  --filter_count 5 \
  cpac_runs/optimized/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                          ┃ Memory    ┃ Memory    ┃ Memory    ┃
┃ Task ID                                  ┃ Used      ┃ Estimated ┃ Efficien… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ cpac_sub-0010042_ses-1.segment_64        │ 1.1376    │ 1.4072    │ 80.84 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.9533    │ 1.1681    │ 81.62 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.7630    │ 0.9104    │ 83.81 %   │
│ cpac_sub-0010042_ses-1.ANTS_T1_to_templ… │ 0.7189    │ 0.9812    │ 73.27 %   │
│ cpac_sub-0010042_ses-1.WM_64.seg_tissue… │ 0.6409    │ 0.8323    │ 77.00 %   │
└──────────────────────────────────────────┴───────────┴───────────┴───────────┘


: 1

In [17]:
cpac parse-resources \
  --filter_field estimate \
  --filter_group lowest \
  --filter_count 5 \
  cpac_runs/optimized/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                          ┃ Memory    ┃ Memory    ┃ Memory    ┃
┃ Task ID                                  ┃ Used      ┃ Estimated ┃ Efficien… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ cpac_sub-0010042_ses-1.CSF_64.seg_tissu… │ 0.1800    │ 0.2223    │ 80.96 %   │
│ cpac_sub-0010042_ses-1.func_ingress_sub… │ 0.1815    │ 0.2245    │ 80.84 %   │
│ cpac_sub-0010042_ses-1.sinker_from-temp… │ 0.2025    │ 0.2245    │ 90.24 %   │
│ cpac_sub-0010042_ses-1.func_ingress_sub… │ 0.1815    │ 0.2245    │ 80.84 %   │
│ cpac_sub-0010042_ses-1.func_ingress_sub… │ 0.1800    │ 0.2245    │ 80.17 %   │
└──────────────────────────────────────────┴───────────┴───────────┴───────────┘


: 1

In [18]:
cpac parse-resources \
  --filter_field runtime \
  --filter_group lowest \
  --filter_count 5 \
  cpac_runs/optimized/log/pipeline_cpac_anat/sub-0010042_ses-1/callback.log

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                                          ┃ Memory    ┃ Memory    ┃ Memory    ┃
┃ Task ID                                  ┃ Used      ┃ Estimated ┃ Efficien… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ cpac_sub-0010042_ses-1.id_string_from-T… │ 0.1762    │ 0.2311    │ 76.28 %   │
│ cpac_sub-0010042_ses-1.json_desc-brain_… │ 0.1762    │ 0.2311    │ 76.28 %   │
│ cpac_sub-0010042_ses-1.id_string_space-… │ 0.1762    │ 0.2343    │ 75.22 %   │
│ cpac_sub-0010042_ses-1.json_desc-reorie… │ 0.1762    │ 0.2257    │ 78.08 %   │
│ cpac_sub-0010042_ses-1.id_string_from-t… │ 0.1762    │ 0.2344    │ 75.18 %   │
└──────────────────────────────────────────┴───────────┴───────────┴───────────┘


: 1

Here are some box-and-whisker plot comparing the efficiency, estimates, and observed memory usage of the four runs we completed above:

![Box-and-whisker plot of efficiency](./images/efficiency.png)
![Box-and-whisker plot of estimated memory usage](./images/estimated.png)
![Box-and-whisker plot of observed memory usage](./images/runtime.png)

## Notes

### Total memory usage
These estimates and observations are per-node. At any given moment, you'll have up to `n_cpus` nodes running at once, plus C-PAC and Nipype will be using some memory, so a worst-case scenario for memory usage is approximately the heaviest `n_cpus` nodes + `1 GB` for overhead.

In this example, we set `n_cpus` to `4`. For the optimized re-run in this example, the four greediest nodes + `1 GB` for overhead = `(1.1376 + 0.9533 + 0.7630 + 0.7189 + 1) GB` = `4.5728 GB`, which is slightly over the specified limit of `4 GB` even though no individual node comes close to overrunning. In most cases, the greediest nodes won't be peaking simultaneously, so we won't expect an out-of-memory error on an optimized run, but they are possible.